# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray


In [ ]:
mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray


In [29]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [30]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [31]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [32]:
# review the collections in our new database
collections = db.list_collection_names()
collections

['establishments']

In [33]:
# Assign the collection to a variable
establishments = db['establishments']

# Find and review one document from the collection
document = establishments.find_one()

# Display the document using pprint
pprint(document)


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650e563183e8ddace47cfd50'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00:

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [34]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [35]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant_data)

In [36]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("The new restaurant has been successfully inserted with ObjectId:", insert_result.inserted_id)
else:
    print("Insertion failed.")

The new restaurant has been successfully inserted with ObjectId: 650e5712816915c8007b659a


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [37]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Define the query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Specify the fields you want to return (BusinessTypeID and BusinessType)
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Find the document that matches the query and project only the specified fields
result = establishments.find_one(query, projection)

# Check if a result was found
if result:
    print("BusinessTypeID:", result["BusinessTypeID"])
    print("BusinessType:", result["BusinessType"])
else:
    print("No matching document found.")


BusinessTypeID: 1
BusinessType: Restaurant/Cafe/Canteen


3. Update the new restaurant with the `BusinessTypeID` you found.

In [38]:
# Update the new restaurant with the correct BusinessTypeID

# Define the filter to identify the restaurant by its BusinessName
filter_query = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the correct BusinessTypeID
update_query = {"$set": {"BusinessTypeID": "YourCorrectBusinessTypeIDHere"}}

# Update the restaurant with the correct BusinessTypeID
update_result = establishments.update_one(filter_query, update_query)

# Check if the update was successful
if update_result.modified_count > 0:
    print("The BusinessTypeID has been updated successfully.")
else:
    print("No matching document found for update.")

The BusinessTypeID has been updated successfully.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [39]:
# Find how many documents have LocalAuthorityName as "Dover"

# Define a query to find documents with LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}

# Use the count_documents method to count the matching documents
dover_count = establishments.count_documents(dover_query)

# Print the count
print("Number of documents with LocalAuthorityName 'Dover':", dover_count)


Number of documents with LocalAuthorityName 'Dover': 994


In [40]:
# Delete all documents where LocalAuthorityName is "Dover"

# Use the delete_many method to remove the matching documents
delete_result = establishments.delete_many(dover_query)

# Get the count of deleted documents
deleted_count = delete_result.deleted_count

# Print the number of deleted documents
print(f"Number of documents with LocalAuthorityName 'Dover' deleted: {deleted_count}")

# Optional: Check the number of remaining documents after deletion
remaining_count = establishments.count_documents({})
print(f"Number of remaining documents in the collection: {remaining_count}")


Number of documents with LocalAuthorityName 'Dover' deleted: 994
Number of remaining documents in the collection: 38786


In [41]:
# Check if any remaining documents include Dover

# Use the count_documents method to count the matching documents
remaining_dover_count = establishments.count_documents(dover_query)

# Check if there are any remaining documents with Dover as LocalAuthorityName
if remaining_dover_count > 0:
    print(f"There are {remaining_dover_count} remaining documents with LocalAuthorityName 'Dover'.")
else:
    print("No remaining documents with LocalAuthorityName 'Dover' found.")

No remaining documents with LocalAuthorityName 'Dover' found.


In [42]:
# Check that other documents remain with 'find_one'

# Use find_one to retrieve one of the remaining documents
remaining_document = establishments.find_one()

# Check if a remaining document was found
if remaining_document:
    # Display the details of the remaining document
    print("Details of one of the remaining documents:")
    pprint(remaining_document)
else:
    print("No remaining documents found.")


Details of one of the remaining documents:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650e563183e8ddace47d0037'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

Use `update_many` to convert `RatingValue` to integer numbers.

In [43]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [44]:
# Change the data type from String to Integer for RatingValue

update_query = [
    {
        "$set": {
            "RatingValue": {
                "$cond": {
                    "if": {
                        "$in": ["$RatingValue", non_ratings]
                    },
                    "then": None,
                    "else": {"$toInt": "$RatingValue"}
                }
            }
        }
    }
]
# Use update_many to apply the update query to all documents
update_result = establishments.update_many({}, update_query)

# Check the number of documents updated
updated_count = update_result.modified_count
print(f"Number of documents updated: {updated_count}")




Number of documents updated: 1
